In [ ]:
nohup python train_nets.py \
--class_number 10572 \
--pretrained_model output/ckpt \
--eval_db_path /workspace/dataset/faces_webface_112x112/ \
--tfrecords_file_path /workspace/dataset/faces_webface_112x112/tfrecords &

In [2]:
import tensorflow as tf

# 定义模型

In [3]:
def inference(inputs):
    x = tf.layers.conv2d(inputs,32,3)
    return x

In [6]:
inputs = tf.placeholder(dtype=tf.float32,shape=(None,28,28,3))

In [7]:
x = inference(inputs)

Instructions for updating:
Colocations handled automatically by placer.


In [19]:
x,inputs

(<tf.Tensor 'conv2d_1/BiasAdd:0' shape=(?, 26, 26, 32) dtype=float32>,
 <tf.Tensor 'Placeholder_1:0' shape=(?, 28, 28, 3) dtype=float32>)

# 模型保存

In [16]:
saver = tf.train.Saver()

with tf.Session() as sess:

    sess.run(tf.global_variables_initializer())

    saver.save(sess,'output/tmp/model.ckpt')

    tf.train.write_graph(sess.graph_def, 'output/tmp/', 'model.pb', as_text=True)

# 使用freeze_graph进行冻结

In [18]:
!freeze_graph --input_graph output/tmp/model.pb \
--input_checkpoint output/tmp/model.ckpt \
--output_graph output/tmp/freeze_model.pb \
--output_node_names conv2d_1/BiasAdd

Instructions for updating:
Use tf.gfile.GFile.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
2019-06-01 10:13:22.699238: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2019-06-01 10:13:22.725859: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3499605000 Hz
2019-06-01 10:13:22.727557: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x562619d33dc0 executing computations on platform Host. Devices:
2019-06-01 10:13:22.727613: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-06-01 10:13:22.839638: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-06-01 10:13:22.841104: I tensorflow/compiler/xl

# 使用tflite_convert进行转换

In [22]:
!tflite_convert --output_file output/tmp/freeze_model.tflite \
--graph_def_file output/tmp/freeze_model.pb \
--output_format TFLITE \
--inference_type FLOAT \
--inference_input_type FLOAT \
--output_arrays conv2d_1/BiasAdd \
--input_arrays Placeholder_1 \
--input_shapes 1,28,28,3

2019-06-01 10:18:24.731131: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2019-06-01 10:18:24.757707: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3499605000 Hz
2019-06-01 10:18:24.759835: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x5569e8195910 executing computations on platform Host. Devices:
2019-06-01 10:18:24.759887: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-06-01 10:18:24.879785: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-06-01 10:18:24.880438: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x5569e808b920 executing computations on platform CUDA. Devices:
2019-06-01 10:18:24.880514: I t

# 测试tflite

In [25]:
import numpy as np
import tensorflow as tf

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path='output/tmp/freeze_model.tflite')
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print('INPUTS: ')
print(input_details)
print('OUTPUTS: ')
print(output_details)

# Test model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=input_details[0]['dtype'])
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])

print('output:')
print(output_data)

INPUTS: 
[{'name': 'Placeholder_1', 'index': 0, 'shape': array([ 1, 28, 28,  3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]
OUTPUTS: 
[{'name': 'conv2d_1/BiasAdd', 'index': 1, 'shape': array([ 1, 26, 26, 32], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]
output:
[[[[ 0.28719145 -0.16600083 -0.11733604 ... -0.15061074 -0.20447142
    -0.27495307]
   [ 0.29369253 -0.2272164  -0.14991726 ... -0.23214409 -0.0111625
    -0.4493868 ]
   [ 0.15429543 -0.1317947  -0.04077595 ... -0.15163535 -0.34605554
    -0.36784413]
   ...
   [ 0.10195651 -0.04193883  0.00162613 ... -0.3153021  -0.28190997
    -0.37845877]
   [ 0.24441911 -0.36807358  0.16235605 ... -0.21481267 -0.17721659
    -0.43001425]
   [ 0.10973265  0.00355946  0.03919911 ... -0.3937525  -0.15404418
    -0.54729503]]

  [[ 0.31108633  0.03298547  0.08658881 ... -0.34510487 -0.22875144
    -0.39901268]
   [ 0.07186887 -0.17360124 -0.08638835 ... -0.25857487 -0.1666781
    -0